### This module runs ANN on TFIDF

In [1]:
#make necessary imports
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
#Read the data
df=pd.read_csv('train.csv')

In [3]:
df['content']=df['title']+df['text']
df=df.dropna()
df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredWhy the Truth...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [4]:
x_train,x_test,y_train,y_test=train_test_split(df['content'], df['label'], test_size=0.2, random_state=7)

In [5]:
#tfidf
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [6]:
tfidf_train

<14628x151206 sparse matrix of type '<class 'numpy.float64'>'
	with 3991333 stored elements in Compressed Sparse Row format>

In [30]:
#building the classifier
def build_classifier():
    clf=Sequential()
    clf.add(Dense(output_dim=20,init='uniform',activation='relu',input_dim=151206))
    clf.add(Dense(output_dim=10,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=5,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

In [33]:
#make necessary imports
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

#build ANN, use k fold cross validation
clf=KerasClassifier(build_fn=build_classifier, batch_size=10, nb_epoch=10)
accuracies=cross_val_score(estimator=clf, X=tfidf_train,y=y_train,cv=10,n_jobs=-1)

In [34]:
#see accuracies
accuracies

array([0.96103895, 0.96445662, 0.57279563, 0.57621324, 0.96240604,
       0.57484621, 0.55707449, 0.56390977, 0.96238029, 0.95896035])

In [32]:
#fit on training data and check accuracies on both test and valid data
clf.fit(tfidf_train,y_train, batch_size=10, nb_epoch=10)
y_test_pred = clf.predict(tfidf_test)
print('algorithm - test dataset accuracy')
print('ANNTFIDF - ' ,round(accuracy_score(y_test, y_test_pred),4))

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=151206, units=20, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=10, kernel_initializer="uniform")`
  """
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=5, kernel_initializer="uniform")`
  
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  import sys


Epoch 1/10
14628/14628 [==============================] - 138s 9ms/step - loss: 0.3528 - accuracy: 0.8790
Epoch 2/10
14628/14628 [==============================] - 176s 12ms/step - loss: 0.1224 - accuracy: 0.9855
Epoch 3/10
14628/14628 [==============================] - 158s 11ms/step - loss: 0.0595 - accuracy: 0.9943
Epoch 4/10
14628/14628 [==============================] - 169s 12ms/step - loss: 0.0353 - accuracy: 0.9962
Epoch 5/10
14628/14628 [==============================] - 168s 11ms/step - loss: 0.0240 - accuracy: 0.9971
Epoch 6/10
14628/14628 [==============================] - 155s 11ms/step - loss: 0.0179 - accuracy: 0.9976
Epoch 7/10
14628/14628 [==============================] - 149s 10ms/step - loss: 0.0147 - accuracy: 0.9979
Epoch 8/10
14628/14628 [==============================] - 157s 11ms/step - loss: 0.0123 - accuracy: 0.9983
Epoch 9/10
14628/14628 [==============================] - 217s 15ms/step - loss: 0.0114 - accuracy: 0.9984
Epoch 10/10
14628/14628 [=============